In [ ]:
import os
import pandas as pd
import matplotlib.dates as mdates
from matplotlib import pyplot as plt
import matplotlib_inline
import seaborn as sns

sns.set(style="whitegrid")
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

# Read in data

In [ ]:
data_dir = os.path.join(os.path.dirname(os.getcwd()), "tutorial_performance_data")

In [ ]:
df = pd.read_csv(os.path.join(data_dir, "all_data.csv"))
df["short_name"] = df["name"].apply(lambda x: x[:21])
df["memory_increase"] = df["max_mem"] - df["start_mem"]
df["datetime"] = pd.to_datetime(df["datetime"])

In [ ]:
df["tmp"] = df.apply(lambda x: x["name"] if x.loc["ran_successfully"] else "", axis=1)
df["ran_successfully"].mean()

In [ ]:
df["name_if_ran"] = df.apply(
    lambda x: x.loc["name"] if x.loc["ran_successfully"] else "", axis=1
)

# Identifies runs where the same set of tutorials completed
# Used to make sure we don't compare aggregated data from
# runs where different tutorials were run
df["set_of_tutorials_run_hash"] = df.groupby("fname")["name_if_ran"].transform(
    lambda x: hash("".join(x.sort_values().values))
)
df.head()

# Plot performance metrics over time

In [ ]:
per_hash_data = (
    df.groupby(["mode", "set_of_tutorials_run_hash"])
    .agg(
        n_runs=("fname", lambda x: len(x.drop_duplicates())),
        most_recent_run=("datetime", "max"),
    )
    .reset_index()
)
# loosen this as more data comes in
per_hash_data["keep"] = (per_hash_data["n_runs"] > 2) | (
    per_hash_data["most_recent_run"] > "2023-02-24"
)
per_hash_data

In [ ]:
hashes_to_plot = per_hash_data.loc[
    per_hash_data["keep"], ["mode", "set_of_tutorials_run_hash"]
]
hashes_to_plot

In [ ]:
time_series_data_wide = (
    df.merge(hashes_to_plot)
    .groupby("fname")
    .agg(
        mode=("mode", "first"),
        set_of_tutorials_run_hash=("set_of_tutorials_run_hash", "first"),
        datetime=("datetime", "first"),
        total_runtime=("runtime", "sum"),
        slowest_runtime=("runtime", "max"),
        avg_memory=("memory_increase", "mean"),
        max_memory=("memory_increase", "max"),
    )
)

time_series_data_wide.head()

In [ ]:
time_series_data_long = pd.melt(
    time_series_data_wide,  # .reset_index(),
    id_vars=[
        # "fname",
        "mode",
        "set_of_tutorials_run_hash",
        "datetime",
    ],
    value_vars=["total_runtime", "slowest_runtime", "avg_memory", "max_memory"],
    var_name="metric",
)
time_series_data_long.dtypes

In [ ]:
def make_time_series_plot(
    time_series_data_long: pd.DataFrame, mode: str
) -> sns.FacetGrid:
    keep_data = time_series_data_long[
        # time_series_data_long["keep"] &
        (time_series_data_long["mode"] == mode)
    ].sort_values("datetime")
    if len(keep_data) == 0:
        print(f"No {mode} data.")
        return
    g = sns.FacetGrid(
        data=keep_data,
        col="metric",
        col_wrap=2,
        aspect=1.5,
        sharey=False,
        hue="set_of_tutorials_run_hash",
        palette="viridis",
    )
    g.map(plt.plot, "datetime", "value")
    g.map(plt.scatter, "datetime", "value")
    g.add_legend()
    g.fig.suptitle(mode)
    plt.subplots_adjust(top=0.9)
    for ax in g.axes.flatten():
        ax.xaxis.set_major_formatter(mdates.DateFormatter("%m-%d"))
    return g

In [ ]:
def make_histogram(time_series_data_long: pd.DataFrame, mode: str) -> sns.FacetGrid:
    if not (per_hash_data["mode"] == mode).any():
        print(f"No relevant {mode} data.")
        return
    most_recent_hash = (
        per_hash_data[per_hash_data["mode"] == mode]
        .sort_values("most_recent_run")["set_of_tutorials_run_hash"]
        .values[-1]
    )

    keep_data = time_series_data_long[
        # time_series_data_long["keep"] &
        (time_series_data_long["mode"] == mode)
        & (time_series_data_long["set_of_tutorials_run_hash"] == most_recent_hash)
    ]
    if len(keep_data) == 0:
        print(f"No relevant {mode} data.")
        return
    g = sns.FacetGrid(
        data=keep_data, col="metric", col_wrap=2, aspect=1.5, sharex=False
    )
    g.map(plt.hist, "value", color="k")
    g.fig.suptitle(mode)
    plt.subplots_adjust(top=0.9)
    return g

In [ ]:
make_time_series_plot(time_series_data_long, "standard")

In [ ]:
make_time_series_plot(time_series_data_long, "smoke-test")

In [ ]:
make_histogram(time_series_data_long, "standard")

In [ ]:
make_histogram(time_series_data_long, "smoke-test")

# Per-tutorial time series plots and histograms

In [ ]:
# time series, one tutorial at a time
def make_per_tutorial_ts_plot(mode: str):
    idx = df["ran_successfully"] & (df["mode"] == mode)
    if not idx.any():
        print(f"No data for {mode}")
        return
    g = sns.FacetGrid(
        data=pd.melt(
            df[idx],
            id_vars=["short_name", "datetime"],
            value_vars=["runtime", "memory_increase"],
            value_name="value",
            var_name="metric",
        ),
        col="metric",
        row="short_name",
        sharey=False,
        aspect=3,
    )
    g.set_titles(col_template="{col_name}", row_template="{row_name}")
    g.map(plt.plot, "datetime", "value", color="k")
    g.map(plt.scatter, "datetime", "value", color="k")

In [ ]:
def make_per_tutorial_hist(mode: str):
    idx = df["ran_successfully"] & (df["mode"] == mode)
    if not idx.any():
        print(f"No data for {mode}")
        return
    g = sns.FacetGrid(
        data=pd.melt(
            df[idx],
            id_vars=["short_name", "datetime"],
            value_vars=["runtime", "memory_increase"],
            value_name="value",
            var_name="metric",
        ),
        col="metric",
        row="short_name",
        sharex=False,
        aspect=3,
    )
    g.set_titles(col_template="{col_name}", row_template="{row_name}")
    g.map(plt.hist, "value", color="k")

In [ ]:
make_per_tutorial_ts_plot("standard")

In [ ]:
make_per_tutorial_ts_plot("smoke-test")

In [ ]:
make_per_tutorial_hist("standard")

In [ ]:
make_per_tutorial_hist("smoke-test")

# Compare tutorials against each other

In [ ]:
last_run_data = df[
    df["datetime"] == df.groupby("mode")["datetime"].transform("max")
].sort_values("memory_increase")
last_run_data_long = pd.melt(
    last_run_data,
    id_vars=["mode", "name"],
    value_vars=["runtime", "memory_increase"],
    var_name="metric",
)
last_run_data_long.head()

In [ ]:
g = sns.catplot(
    data=last_run_data_long,
    col="metric",
    row="mode",
    x="value",
    y="name",
    kind="bar",
    orient="h",
    sharex=False,
    color="k",
)
g.fig.suptitle("Metrics on the most recent run in this mode")
plt.subplots_adjust(top=0.9)